# Module d'importation

Importe et crée les groupes associés, plus deux groupes de base pour savoir ce qu'on manipule

In [2]:
import numpy as np
import pandas as pd
    
training_data = pd.read_csv(r'C:\Users\Admin\Desktop\Challenge data\data_challenge_ens-master\Training_XS_300.csv', delimiter=';')
training_data.apply(pd.to_numeric)
training_group=training_data.loc[:,['ID', 'bid_1', 'bid_size_1','nb_trade']].groupby('ID')

target_data = pd.read_csv(r'C:\Users\Admin\Desktop\Challenge data\data_challenge_ens-master\Target_data.csv', delimiter=';', nrows=len(training_group))
target_data.apply(pd.to_numeric)
target_group=target_data.groupby('ID')
training_group1=training_group.get_group(1)
target_group1=target_group.get_group(1)

# Fonctions type

Pour itérer sur les groupes, il est possible d'utiliser des for-loops, a priori il faut préférer des variantes de map, en particulier apply(), le problème c'est qu'apply renvoie un DataFrame qui sera donc une copie de la data d'origine. Il n'est pas possible de modifier en place le DataFrame qui a généré le groupe. C'est bien adapté pour des sum() ou std() (catégorie agg) moins pour ajouter une colonne (catégorie transform)

Self-modification, ce type de fonction s'itère bien avec apply().

In [3]:
def add_product(vect, ind1, ind2):
    ## adds a new colum to vect from the product of ind1 and ind2
    vect[str(ind1)+"_prod_"+str(ind2)]=vect[str(ind2)]*vect[str(ind2)]
    return vect

In [4]:
transf_training_data=training_group.apply(lambda x: add_product(x, 'bid_1', 'bid_size_1'))

Joint-modification, c'est là que ça coince. Il est facile d'appliquer une fonction conjointe sur deux data frame, pas pour deux groupes. Un groupe ne sert que comme objet sur lequel faire des apply ou des opérations partielles. Il n'est pas un objet manipulable en soi.

In [5]:
def add_variation_counter_vect(vect1, vect2, ind1):
    ## adds a new colum to vect2 from the variation counter of ind1
    vect2[str(ind1)+"_varcount"]=vect1[str(ind1)].diff().astype(bool).sum()-1
    return vect2

In [6]:
transf_target1=add_variation_counter_vect(training_group1, target_group1, 'nb_trade')

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


## Suggestions

Je ne pense pas que manipuler deux groupes conjointement soit une bonne idée. Il faudrait faire des opérations intra-groupe puis des actions inter-DataFrame. Il y a un problème d'indexation chez nous parce que nous avons laissé celle naturelle d'Excel qui compte les lignes. Celle-ci est inutile, je propose de la remplacer par une indexation hiérarchique à deux niveaux 'ID' puis 'offset' dont les valeurs d'index seraient des float. Les autres colonnes resteraient telles quelles avec possibilité de faire une séparation "old/new" si on crée des variables.

L'autre problème vient de scikit, il attend une matrice nb_points x nb_features. Dans tous les cas il faudra déplier nos dataframe par reshape ou unstack (si hiérarchique).

### Example scikit

On applique un SVC (SVM pour classes avec noyau gaussien ici) très facilement et on récupère l'accuracy. Le gros problème est que cette méthode s'adapte mal à l'échelle. D'où grosse réflexion sur les méthodes à employer.

In [7]:
nbt=training_data['nb_trade']
nbt=np.reshape(nbt,(-1,8))
targ=target_data['TARGET']

from sklearn import svm
clf = svm.SVC()
clf.fit(nbt, targ)  
clf.score(nbt, targ)

0.76530612244897955